# Query JCloud

This notebook connects to the Flow that was created in the notebook "1. JCloud Deployment" and gets the results necessary for the evaluation by iterating over the set of queries.

## Install and import libraries 

In [ ]:
!pip install jina
!pip install docarray

In [ ]:
from jina import Client
from docarray import DocumentArray, Document
import numpy as np
import pandas as pd
import json

## Log into JCloud

In [ ]:
!jc login
!jc list

## Read the queries from the data folder and query Jiina flow with each query

Do not forget to insert the right host name!

In [ ]:
client = Client(host="grpcs://\<your-flow-name\>-\<your-flow-id\>.jina.ai")

df_relevance = pd.DataFrame()

df_queries = pd.read_csv('../data/queries.txt', sep="\t", header=None)

for query in df_queries.itertuples():
    
    text_query = Document(text=query[2])
    jcloud_matches = client.search(text_query, show_progress=True)
    for i in range (0, 10):
        # create a new row for the DataFrame and append it
        row = { 'query_id' : str(query[1]), 'Q0' : "Q0", 'product_id' : jcloud_matches[0].matches[i].tags['id'], 'position' : str(i), 'relevance' : jcloud_matches[0].matches[i].scores['cosine'].value, 'run': 'jina' }
        df_relevance = df_relevance.append(row, ignore_index=True)
    
# store the DataFrame without header and index, with tabs as delimiters
name = '../data/jina_result.qrels'
df_relevance.to_csv(name, sep="\t", header=False, index=False)
df_relevance
# work with two counters:
# 1) one to iterate over the list of queries and have a query id instead of a query
# 2) another one to iterate over the result sets to have the position of the result in the result set 

# DataFrame with columns:
# query_id: the id of the query as the trec_eval tool needs a numeric id rather than a query string as an identifier
# Q0: all lines have Q0, currently unused by trec_eval
# product_id: the id of the product in the hit list
# position: the position of the product in the result set
# relevance: relevance as given by the search engine, cosine similarity in the case of jina
# run: the name of the query run

### Sample with two queries

In [ ]:
jcloud_matches[0].matches[0].tags['id']

In [ ]:
df_queries = pd.read_csv('../data/queries.txt', sep="\t", header=None)

for query in df_queries.itertuples():
    
    text_query = Document(text=query[2])
    jcloud_matches = client.search(text_query, show_progress=True)
    for i in range (0, 10):
        # create a new row for the DataFrame and append it
        row = { 'query_id' : str(query_i), 'Q0' : "Q0", 'product_id' : jcloud_matches[0].matches[i].tags['id'], 'position' : str(i), 'relevance' : jcloud_matches[0].matches[i].scores['cosine'].value, 'run': 'jina' }
        df_relevance = df_relevance.append(row, ignore_index=True)
    
df_relevance
